In [1]:
import sys
sys.path.insert(0, '/home/knoop/local/hilde')

import importlib as il


import numpy as np

from matplotlib import pyplot as plt

from hilde.trajectory import reader
from hilde.konstanten import omega_to_THz

from hilde.harmonic_analysis import parse_tdep_forceconstant, HarmonicAnalysis

from hilde.harmonic_analysis.displacements import get_U

from hilde.harmonic_analysis.normal_modes import u_I_to_u_s, map_L_to_i

from hilde.helpers import progressbar

In [2]:
traj = reader('no_drift.yaml')

.. no_drift.yaml parsed in 3.942s


In [3]:
fcs, lps = parse_tdep_forceconstant('outfile.forceconstant')

ha = HarmonicAnalysis(traj.primitive, traj.supercell, fcs, lps)

Parse force constants from
  outfile.forceconstant
.. remap representation for supercell:  False
.. Number of atoms:   2
.. Real space cutoff: 3.660 \AA
.. Number of lattice points: 54 (33 unique)
.. time elapsed: 0.004s
Set up harmonic analysis for MgO:
.. found 32 (31) lattice points in 0.018s
.. matched 64 positions in supercell and primitive cell in 0.111s
.. found 32 (31) lattice points in 0.002s
.. time elapsed: 0.389s


In [4]:
N = len(traj)
atoms0 = traj.supercell
kwargs = {'q_points': ha.q_points, 'lattice_points': ha.lattice_points_supercell, 'eigenvectors': ha.eigenvectors,
        'indeces': ha.I_to_iL}

In [5]:
u_I = get_U(traj[1], atoms0)

iL = ha.I_to_iL
RLs = ha.lattice_points_supercell[ha.I_to_iL[:, 1]]
RLs.shape

(64, 3)

In [6]:
na = len(ha.supercell)
nq, ns = ha.eigenvectors.shape[:2]
ievs = ha.eigenvectors.conj().swapaxes(1, 2).reshape(nq, ns, ns // 3, 3)

ievs.shape, ievs[:, :, ha.I_to_iL[:, 0], :].shape, (ha.q_points @ RLs.T).repeat(3, axis=1).shape

((32, 6, 2, 3), (32, 6, 64, 3), (32, 192))

In [7]:
aug_ev = 1/(nq**.5) * np.exp(-2j * np.pi * (ha.q_points @ RLs.T).repeat(3, axis=1).reshape(nq, na, -1))[:, None, :, :] *  ievs[:, :, ha.I_to_iL[:, 0], :]

In [8]:
%%timeit
us =  (aug_ev * u_I).sum(axis=(2, 3))

270 µs ± 46.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [9]:
%%timeit
usp = u_I_to_u_s(u_I, **kwargs)

1.63 ms ± 168 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
# are they equal?
us =  (aug_ev * u_I).sum(axis=(2, 3))
usp = u_I_to_u_s(u_I, **kwargs)
np.linalg.norm(usp - us)

1.7434926320724184e-15